In [1]:
# example of a gan for generating faces
from numpy import load
from numpy import zeros
from numpy import ones
from numpy.random import randn
from numpy.random import randint
from keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers import Flatten
from keras.layers import Conv2D
from keras.layers import Conv2DTranspose
from keras.layers import LeakyReLU
from keras.layers import Dropout
from matplotlib import pyplot

# define the standalone discriminator model
def define_discriminator(in_shape=(80,80,3)):
    model = Sequential()
    # normal
    model.add(Conv2D(128, (5,5), padding='same', input_shape=in_shape))
    model.add(LeakyReLU(alpha=0.2))
    # downsample to 40x40
    model.add(Conv2D(128, (5,5), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    # downsample to 20x30
    model.add(Conv2D(128, (5,5), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    # downsample to 10x10
    model.add(Conv2D(128, (5,5), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    # downsample to 5x5
    model.add(Conv2D(128, (5,5), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    # classifier
    model.add(Flatten())
    model.add(Dropout(0.4))
    model.add(Dense(1, activation='sigmoid'))
    # compile model
    opt = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    return model

# define the standalone generator model
def define_generator(latent_dim):
    model = Sequential()
    # foundation for 5x5 feature maps
    n_nodes = 128 * 5 * 5
    model.add(Dense(n_nodes, input_dim=latent_dim))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Reshape((5, 5, 128)))
    # upsample to 10x10
    model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    # upsample to 20x20
    model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    # upsample to 40x40
    model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    # upsample to 80x80
    model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    # output layer 80x80x3
    model.add(Conv2D(3, (5,5), activation='tanh', padding='same'))
    return model

# define the combined generator and discriminator model, for updating the generator
def define_gan(g_model, d_model):
    # make weights in the discriminator not trainable
    d_model.trainable = False
    # connect them
    model = Sequential()
    # add generator
    model.add(g_model)
    # add the discriminator
    model.add(d_model)
    # compile model
    opt = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt)
    return model

# load and prepare training images
def load_real_samples():
    # load the face dataset
    data = load('img_align_celeba.npz')
    X = data['arr_0']
    # convert from unsigned ints to floats
    X = X.astype('float32')
    # scale from [0,255] to [-1,1]
    X = (X - 127.5) / 127.5
    return X

# select real samples
def generate_real_samples(dataset, n_samples):
    # choose random instances
    ix = randint(0, dataset.shape[0], n_samples)
    # retrieve selected images
    X = dataset[ix]
    # generate 'real' class labels (1)
    y = ones((n_samples, 1))
    return X, y

# generate points in latent space as input for the generator
def generate_latent_points(latent_dim, n_samples):
    # generate points in the latent space
    x_input = randn(latent_dim * n_samples)
    # reshape into a batch of inputs for the network
    x_input = x_input.reshape(n_samples, latent_dim)
    return x_input

# use the generator to generate n fake examples, with class labels
def generate_fake_samples(g_model, latent_dim, n_samples):
    # generate points in latent space
    x_input = generate_latent_points(latent_dim, n_samples)
    # predict outputs
    X = g_model.predict(x_input)
    # create 'fake' class labels (0)
    y = zeros((n_samples, 1))
    return X, y

# create and save a plot of generated images
def save_plot(examples, epoch, n=10):
    # scale from [-1,1] to [0,1]
    examples = (examples + 1) / 2.0
    # plot images
    for i in range(n * n):
        # define subplot
        pyplot.subplot(n, n, 1 + i)
        # turn off axis
        pyplot.axis('off')
        # plot raw pixel data
        pyplot.imshow(examples[i])
    # save plot to file
    filename = 'generated_plot_e%03d.png' % (epoch+1)
    pyplot.savefig(filename)
    pyplot.close()

# evaluate the discriminator, plot generated images, save generator model
def summarize_performance(epoch, g_model, d_model, dataset, latent_dim, n_samples=100):
    # prepare real samples
    X_real, y_real = generate_real_samples(dataset, n_samples)
    # evaluate discriminator on real examples
    _, acc_real = d_model.evaluate(X_real, y_real, verbose=0)
    # prepare fake examples
    x_fake, y_fake = generate_fake_samples(g_model, latent_dim, n_samples)
    # evaluate discriminator on fake examples
    _, acc_fake = d_model.evaluate(x_fake, y_fake, verbose=0)
    # summarize discriminator performance
    print('>Accuracy real: %.0f%%, fake: %.0f%%' % (acc_real*100, acc_fake*100))
    # save plot
    save_plot(x_fake, epoch)
    # save the generator model tile file
    filename = 'generator_model_%03d.h5' % (epoch+1)
    g_model.save(filename)

# train the generator and discriminator
def train(g_model, d_model, gan_model, dataset, latent_dim, n_epochs=100, n_batch=128):
    bat_per_epo = dataset.shape[0] // n_batch
    half_batch = n_batch // 2
    # manually enumerate epochs
    for i in range(n_epochs):
        # enumerate batches over the training set
        for j in range(bat_per_epo):
            # get randomly selected 'real' samples
            X_real, y_real = generate_real_samples(dataset, half_batch)
            # update discriminator model weights
            d_loss1, _ = d_model.train_on_batch(X_real, y_real)
            # generate 'fake' examples
            X_fake, y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
            # update discriminator model weights
            d_loss2, _ = d_model.train_on_batch(X_fake, y_fake)
            # prepare points in latent space as input for the generator
            X_gan = generate_latent_points(latent_dim, n_batch)
            # create inverted labels for the fake samples
            y_gan = ones((n_batch, 1))
            # update the generator via the discriminator's error
            g_loss = gan_model.train_on_batch(X_gan, y_gan)
            # summarize loss on this batch
            print('>%d, %d/%d, d1=%.3f, d2=%.3f g=%.3f' %
                (i+1, j+1, bat_per_epo, d_loss1, d_loss2, g_loss))
        # evaluate the model performance, sometimes
        if (i+1) % 10 == 0:
            summarize_performance(i, g_model, d_model, dataset, latent_dim)

# size of the latent space
latent_dim = 200
# create the discriminator
d_model = define_discriminator()
# create the generator
g_model = define_generator(latent_dim)
# create the gan
gan_model = define_gan(g_model, d_model)
# load image data
dataset = load_real_samples()
# train model
train(g_model, d_model, gan_model, dataset, latent_dim)

>1, 1/78, d1=0.684, d2=0.696 g=0.692
>1, 2/78, d1=0.495, d2=0.704 g=0.683
>1, 3/78, d1=0.135, d2=0.756 g=0.641
>1, 4/78, d1=0.007, d2=0.840 g=0.638
>1, 5/78, d1=0.014, d2=0.806 g=0.665
>1, 6/78, d1=0.048, d2=0.759 g=0.686
>1, 7/78, d1=0.047, d2=0.716 g=0.708
>1, 8/78, d1=0.051, d2=0.681 g=0.727
>1, 9/78, d1=0.004, d2=0.660 g=0.796
>1, 10/78, d1=0.009, d2=0.606 g=0.980
>1, 11/78, d1=0.005, d2=0.504 g=1.313
>1, 12/78, d1=0.023, d2=0.446 g=1.346
>1, 13/78, d1=0.000, d2=0.376 g=1.618
>1, 14/78, d1=0.230, d2=0.388 g=1.229
>1, 15/78, d1=0.000, d2=0.346 g=1.489
>1, 16/78, d1=0.000, d2=0.233 g=2.059
>1, 17/78, d1=0.000, d2=0.130 g=3.009
>1, 18/78, d1=0.000, d2=0.052 g=4.425
>1, 19/78, d1=0.000, d2=0.014 g=6.083
>1, 20/78, d1=0.000, d2=0.005 g=6.872
>1, 21/78, d1=0.000, d2=0.070 g=12.161
>1, 22/78, d1=0.849, d2=0.010 g=5.002
>1, 23/78, d1=0.000, d2=0.617 g=8.530
>1, 24/78, d1=0.000, d2=0.000 g=10.543
>1, 25/78, d1=0.000, d2=0.000 g=9.135
>1, 26/78, d1=0.000, d2=0.000 g=7.739
>1, 27/78, d1=0.000

>3, 61/78, d1=0.274, d2=0.285 g=2.592
>3, 62/78, d1=0.318, d2=0.340 g=2.503
>3, 63/78, d1=0.255, d2=0.272 g=2.351
>3, 64/78, d1=0.256, d2=0.270 g=2.145
>3, 65/78, d1=0.213, d2=0.203 g=2.402
>3, 66/78, d1=0.139, d2=0.152 g=2.425
>3, 67/78, d1=0.109, d2=0.148 g=2.577
>3, 68/78, d1=0.052, d2=0.113 g=2.839
>3, 69/78, d1=0.110, d2=0.117 g=2.704
>3, 70/78, d1=0.074, d2=0.142 g=2.970
>3, 71/78, d1=0.059, d2=0.160 g=3.384
>3, 72/78, d1=0.230, d2=0.252 g=2.916
>3, 73/78, d1=0.124, d2=0.294 g=3.064
>3, 74/78, d1=0.180, d2=0.242 g=3.028
>3, 75/78, d1=0.197, d2=0.208 g=3.141
>3, 76/78, d1=0.207, d2=0.220 g=2.979
>3, 77/78, d1=0.104, d2=0.166 g=2.997
>3, 78/78, d1=0.065, d2=0.116 g=3.103
>4, 1/78, d1=0.125, d2=0.210 g=3.033
>4, 2/78, d1=0.048, d2=0.138 g=3.566
>4, 3/78, d1=0.094, d2=0.080 g=3.861
>4, 4/78, d1=0.037, d2=0.084 g=4.329
>4, 5/78, d1=0.089, d2=0.089 g=4.209
>4, 6/78, d1=0.185, d2=0.328 g=4.652
>4, 7/78, d1=0.167, d2=0.217 g=3.716
>4, 8/78, d1=0.671, d2=0.950 g=2.474
>4, 9/78, d1=0.065, 

>6, 44/78, d1=0.489, d2=0.078 g=3.295
>6, 45/78, d1=0.147, d2=0.221 g=3.386
>6, 46/78, d1=0.108, d2=0.108 g=3.912
>6, 47/78, d1=0.108, d2=0.063 g=3.838
>6, 48/78, d1=0.182, d2=0.137 g=3.564
>6, 49/78, d1=0.074, d2=0.119 g=4.370
>6, 50/78, d1=0.157, d2=0.102 g=3.563
>6, 51/78, d1=0.224, d2=0.392 g=4.142
>6, 52/78, d1=0.068, d2=0.291 g=5.109
>6, 53/78, d1=0.698, d2=0.988 g=5.285
>6, 54/78, d1=1.710, d2=0.197 g=2.044
>6, 55/78, d1=0.215, d2=0.651 g=1.944
>6, 56/78, d1=0.205, d2=0.368 g=2.588
>6, 57/78, d1=0.265, d2=0.269 g=2.615
>6, 58/78, d1=0.378, d2=0.338 g=2.673
>6, 59/78, d1=0.286, d2=0.247 g=2.864
>6, 60/78, d1=0.278, d2=0.195 g=2.934
>6, 61/78, d1=0.214, d2=0.208 g=2.868
>6, 62/78, d1=0.264, d2=0.275 g=3.563
>6, 63/78, d1=0.374, d2=0.228 g=3.586
>6, 64/78, d1=0.328, d2=0.335 g=4.323
>6, 65/78, d1=0.717, d2=0.147 g=2.728
>6, 66/78, d1=0.175, d2=0.253 g=2.880
>6, 67/78, d1=0.130, d2=0.115 g=3.139
>6, 68/78, d1=0.224, d2=0.136 g=3.006
>6, 69/78, d1=0.207, d2=0.150 g=3.014
>6, 70/78, d

>9, 27/78, d1=0.183, d2=0.084 g=4.206
>9, 28/78, d1=0.096, d2=0.126 g=4.366
>9, 29/78, d1=0.125, d2=0.097 g=4.015
>9, 30/78, d1=0.081, d2=0.145 g=4.557
>9, 31/78, d1=0.150, d2=0.155 g=5.051
>9, 32/78, d1=0.187, d2=0.119 g=5.085
>9, 33/78, d1=0.159, d2=0.215 g=6.115
>9, 34/78, d1=0.350, d2=0.134 g=4.607
>9, 35/78, d1=0.144, d2=0.698 g=11.618
>9, 36/78, d1=3.579, d2=0.507 g=2.789
>9, 37/78, d1=0.171, d2=0.244 g=2.677
>9, 38/78, d1=0.163, d2=0.194 g=2.907
>9, 39/78, d1=0.180, d2=0.172 g=3.404
>9, 40/78, d1=0.162, d2=0.085 g=3.268
>9, 41/78, d1=0.134, d2=0.106 g=3.134
>9, 42/78, d1=0.099, d2=0.124 g=3.055
>9, 43/78, d1=0.116, d2=0.134 g=2.973
>9, 44/78, d1=0.063, d2=0.110 g=3.132
>9, 45/78, d1=0.233, d2=0.262 g=3.385
>9, 46/78, d1=0.162, d2=0.099 g=3.696
>9, 47/78, d1=0.194, d2=0.110 g=3.310
>9, 48/78, d1=0.081, d2=0.077 g=3.463
>9, 49/78, d1=0.105, d2=0.110 g=3.284
>9, 50/78, d1=0.048, d2=0.125 g=3.299
>9, 51/78, d1=0.104, d2=0.173 g=3.033
>9, 52/78, d1=0.179, d2=0.185 g=2.708
>9, 53/78, 

>12, 5/78, d1=0.131, d2=0.160 g=4.585
>12, 6/78, d1=0.366, d2=0.157 g=3.594
>12, 7/78, d1=0.084, d2=0.503 g=6.613
>12, 8/78, d1=0.434, d2=0.067 g=5.176
>12, 9/78, d1=0.148, d2=0.187 g=4.267
>12, 10/78, d1=0.192, d2=0.184 g=4.243
>12, 11/78, d1=0.389, d2=0.125 g=3.410
>12, 12/78, d1=0.298, d2=0.219 g=2.960
>12, 13/78, d1=0.250, d2=0.228 g=3.200
>12, 14/78, d1=0.276, d2=0.153 g=3.133
>12, 15/78, d1=0.320, d2=0.555 g=3.887
>12, 16/78, d1=0.359, d2=0.137 g=4.318
>12, 17/78, d1=0.376, d2=0.267 g=4.112
>12, 18/78, d1=0.198, d2=0.103 g=5.377
>12, 19/78, d1=0.284, d2=0.140 g=5.713
>12, 20/78, d1=0.153, d2=0.074 g=4.588
>12, 21/78, d1=0.042, d2=0.333 g=5.506
>12, 22/78, d1=0.348, d2=0.108 g=4.029
>12, 23/78, d1=0.259, d2=0.331 g=4.376
>12, 24/78, d1=0.311, d2=0.172 g=4.320
>12, 25/78, d1=0.416, d2=0.244 g=4.036
>12, 26/78, d1=0.172, d2=0.279 g=4.781
>12, 27/78, d1=0.354, d2=0.065 g=3.678
>12, 28/78, d1=0.231, d2=0.196 g=3.074
>12, 29/78, d1=0.185, d2=0.236 g=3.648
>12, 30/78, d1=0.186, d2=0.143

...

>87, 56/78, d1=0.587, d2=0.399 g=2.642
>87, 57/78, d1=0.417, d2=0.415 g=2.877
>87, 58/78, d1=0.367, d2=0.309 g=2.969
>87, 59/78, d1=0.450, d2=0.358 g=2.673
>87, 60/78, d1=0.256, d2=0.438 g=2.790
>87, 61/78, d1=0.567, d2=0.337 g=2.343
>87, 62/78, d1=0.273, d2=0.421 g=2.601
>87, 63/78, d1=0.283, d2=0.427 g=2.961
>87, 64/78, d1=0.345, d2=0.215 g=2.826
>87, 65/78, d1=0.427, d2=0.366 g=2.123
>87, 66/78, d1=0.188, d2=0.491 g=3.091
>87, 67/78, d1=0.472, d2=0.276 g=2.925
>87, 68/78, d1=0.319, d2=0.330 g=2.741
>87, 69/78, d1=0.219, d2=0.356 g=3.076
>87, 70/78, d1=0.391, d2=0.447 g=2.978
>87, 71/78, d1=0.329, d2=0.333 g=2.846
>87, 72/78, d1=0.296, d2=0.277 g=2.596
>87, 73/78, d1=0.349, d2=0.323 g=2.659
>87, 74/78, d1=0.309, d2=0.423 g=3.152
>87, 75/78, d1=0.483, d2=0.344 g=2.650
>87, 76/78, d1=0.287, d2=0.475 g=2.859
>87, 77/78, d1=0.607, d2=0.390 g=3.078
>87, 78/78, d1=0.469, d2=0.300 g=2.501
>88, 1/78, d1=0.357, d2=0.397 g=2.486
>88, 2/78, d1=0.244, d2=0.349 g=2.728
>88, 3/78, d1=0.340, d2=0.4

>90, 33/78, d1=0.400, d2=0.398 g=3.079
>90, 34/78, d1=0.388, d2=0.482 g=3.341
>90, 35/78, d1=0.414, d2=0.282 g=3.113
>90, 36/78, d1=0.319, d2=0.550 g=3.373
>90, 37/78, d1=0.544, d2=0.600 g=3.672
>90, 38/78, d1=0.568, d2=0.177 g=2.634
>90, 39/78, d1=0.404, d2=0.508 g=2.573
>90, 40/78, d1=0.381, d2=0.428 g=2.797
>90, 41/78, d1=0.479, d2=0.248 g=2.349
>90, 42/78, d1=0.273, d2=0.463 g=2.814
>90, 43/78, d1=0.330, d2=0.201 g=2.849
>90, 44/78, d1=0.413, d2=0.280 g=2.545
>90, 45/78, d1=0.243, d2=0.445 g=2.984
>90, 46/78, d1=0.439, d2=0.366 g=2.945
>90, 47/78, d1=0.306, d2=0.329 g=2.996
>90, 48/78, d1=0.432, d2=0.326 g=2.611
>90, 49/78, d1=0.355, d2=0.381 g=2.724
>90, 50/78, d1=0.445, d2=0.428 g=3.040
>90, 51/78, d1=0.377, d2=0.311 g=2.883
>90, 52/78, d1=0.340, d2=0.391 g=3.153
>90, 53/78, d1=0.486, d2=0.284 g=3.108
>90, 54/78, d1=0.435, d2=0.496 g=3.058
>90, 55/78, d1=0.194, d2=0.190 g=3.094
>90, 56/78, d1=0.493, d2=0.509 g=2.724
>90, 57/78, d1=0.384, d2=0.619 g=3.295
>90, 58/78, d1=0.611, d2=

>93, 9/78, d1=0.446, d2=0.401 g=3.113
>93, 10/78, d1=0.359, d2=0.462 g=3.367
>93, 11/78, d1=0.636, d2=0.375 g=2.626
>93, 12/78, d1=0.358, d2=0.629 g=3.049
>93, 13/78, d1=0.311, d2=0.238 g=3.176
>93, 14/78, d1=0.426, d2=0.402 g=2.594
>93, 15/78, d1=0.370, d2=0.436 g=3.085
>93, 16/78, d1=0.613, d2=0.411 g=2.761
>93, 17/78, d1=0.416, d2=0.430 g=2.543
>93, 18/78, d1=0.320, d2=0.391 g=2.692
>93, 19/78, d1=0.365, d2=0.306 g=2.697
>93, 20/78, d1=0.299, d2=0.225 g=2.895
>93, 21/78, d1=0.358, d2=0.582 g=3.238
>93, 22/78, d1=0.405, d2=0.363 g=3.558
>93, 23/78, d1=0.565, d2=0.369 g=3.035
>93, 24/78, d1=0.462, d2=0.465 g=3.318
>93, 25/78, d1=0.308, d2=0.217 g=3.036
>93, 26/78, d1=0.422, d2=0.604 g=3.311
>93, 27/78, d1=0.404, d2=0.277 g=3.200
>93, 28/78, d1=0.413, d2=0.463 g=2.558
>93, 29/78, d1=0.459, d2=0.444 g=2.475
>93, 30/78, d1=0.417, d2=0.296 g=2.378
>93, 31/78, d1=0.487, d2=0.491 g=2.586
>93, 32/78, d1=0.342, d2=0.330 g=2.559
>93, 33/78, d1=0.349, d2=0.262 g=2.582
>93, 34/78, d1=0.347, d2=0

>95, 64/78, d1=0.396, d2=0.389 g=2.607
>95, 65/78, d1=0.284, d2=0.457 g=3.214
>95, 66/78, d1=0.471, d2=0.346 g=2.948
>95, 67/78, d1=0.607, d2=0.599 g=2.668
>95, 68/78, d1=0.357, d2=0.387 g=2.898
>95, 69/78, d1=0.306, d2=0.273 g=2.741
>95, 70/78, d1=0.381, d2=0.331 g=2.319
>95, 71/78, d1=0.246, d2=0.303 g=2.760
>95, 72/78, d1=0.381, d2=0.510 g=2.763
>95, 73/78, d1=0.460, d2=0.264 g=2.670
>95, 74/78, d1=0.363, d2=0.373 g=2.446
>95, 75/78, d1=0.255, d2=0.412 g=3.047
>95, 76/78, d1=0.446, d2=0.236 g=2.796
>95, 77/78, d1=0.453, d2=0.662 g=2.835
>95, 78/78, d1=0.439, d2=0.227 g=2.991
>96, 1/78, d1=0.271, d2=0.280 g=2.517
>96, 2/78, d1=0.371, d2=0.421 g=2.623
>96, 3/78, d1=0.291, d2=0.431 g=3.068
>96, 4/78, d1=0.641, d2=0.361 g=2.659
>96, 5/78, d1=0.454, d2=0.361 g=2.360
>96, 6/78, d1=0.256, d2=0.463 g=2.371
>96, 7/78, d1=0.316, d2=0.259 g=2.661
>96, 8/78, d1=0.548, d2=0.378 g=2.299
>96, 9/78, d1=0.439, d2=0.431 g=2.344
>96, 10/78, d1=0.327, d2=0.387 g=2.568
>96, 11/78, d1=0.356, d2=0.323 g=2

>98, 41/78, d1=0.340, d2=0.386 g=2.609
>98, 42/78, d1=0.345, d2=0.401 g=2.889
>98, 43/78, d1=0.283, d2=0.249 g=3.144
>98, 44/78, d1=0.200, d2=0.262 g=3.107
>98, 45/78, d1=0.259, d2=0.354 g=3.083
>98, 46/78, d1=0.396, d2=0.454 g=2.900
>98, 47/78, d1=0.405, d2=0.514 g=3.163
>98, 48/78, d1=0.501, d2=0.328 g=2.970
>98, 49/78, d1=0.439, d2=0.308 g=2.445
>98, 50/78, d1=0.338, d2=0.319 g=2.545
>98, 51/78, d1=0.268, d2=0.364 g=2.920
>98, 52/78, d1=0.323, d2=0.282 g=3.026
>98, 53/78, d1=0.198, d2=0.224 g=3.129
>98, 54/78, d1=0.276, d2=0.453 g=3.084
>98, 55/78, d1=0.462, d2=0.431 g=2.812
>98, 56/78, d1=0.399, d2=0.388 g=2.866
>98, 57/78, d1=0.352, d2=0.413 g=2.843
>98, 58/78, d1=0.283, d2=0.302 g=2.850
>98, 59/78, d1=0.378, d2=0.333 g=2.806
>98, 60/78, d1=0.407, d2=0.500 g=3.132
>98, 61/78, d1=0.269, d2=0.203 g=3.025
>98, 62/78, d1=0.381, d2=0.685 g=3.170
>98, 63/78, d1=0.646, d2=0.461 g=3.146
>98, 64/78, d1=0.386, d2=0.311 g=2.996
>98, 65/78, d1=0.343, d2=0.305 g=2.648
>98, 66/78, d1=0.391, d2=

In [3]:
train(g_model, d_model, gan_model, dataset, latent_dim)

>1, 1/78, d1=0.339, d2=0.455 g=2.851
>1, 2/78, d1=0.580, d2=0.380 g=2.578
>1, 3/78, d1=0.382, d2=0.403 g=2.422
>1, 4/78, d1=0.375, d2=0.355 g=2.455
>1, 5/78, d1=0.274, d2=0.510 g=2.654
>1, 6/78, d1=0.349, d2=0.286 g=2.633
>1, 7/78, d1=0.497, d2=0.431 g=2.160
>1, 8/78, d1=0.250, d2=0.354 g=2.471
>1, 9/78, d1=0.253, d2=0.295 g=2.519
>1, 10/78, d1=0.334, d2=0.450 g=2.614
>1, 11/78, d1=0.407, d2=0.315 g=2.963
>1, 12/78, d1=0.511, d2=0.630 g=2.699
>1, 13/78, d1=0.426, d2=0.309 g=2.751
>1, 14/78, d1=0.293, d2=0.503 g=3.115
>1, 15/78, d1=0.525, d2=0.311 g=2.641
>1, 16/78, d1=0.443, d2=0.648 g=2.548
>1, 17/78, d1=0.558, d2=0.322 g=2.638
>1, 18/78, d1=0.318, d2=0.451 g=2.813
>1, 19/78, d1=0.432, d2=0.435 g=2.666
>1, 20/78, d1=0.359, d2=0.318 g=2.904
>1, 21/78, d1=0.448, d2=0.353 g=2.788
>1, 22/78, d1=0.342, d2=0.362 g=2.600
>1, 23/78, d1=0.308, d2=0.358 g=2.922
>1, 24/78, d1=0.326, d2=0.316 g=2.937
>1, 25/78, d1=0.312, d2=0.194 g=2.540
>1, 26/78, d1=0.253, d2=0.546 g=2.926
>1, 27/78, d1=0.369, 

>3, 62/78, d1=0.325, d2=0.400 g=2.439
>3, 63/78, d1=0.270, d2=0.282 g=2.468
>3, 64/78, d1=0.290, d2=0.427 g=2.552
>3, 65/78, d1=0.232, d2=0.263 g=2.787
>3, 66/78, d1=0.429, d2=0.322 g=2.461
>3, 67/78, d1=0.410, d2=0.589 g=2.822
>3, 68/78, d1=0.377, d2=0.324 g=3.114
>3, 69/78, d1=0.396, d2=0.287 g=2.608
>3, 70/78, d1=0.319, d2=0.410 g=2.791
>3, 71/78, d1=0.290, d2=0.341 g=3.128
>3, 72/78, d1=0.413, d2=0.302 g=2.605
>3, 73/78, d1=0.224, d2=0.344 g=2.931
>3, 74/78, d1=0.473, d2=0.518 g=3.117
>3, 75/78, d1=0.378, d2=0.448 g=3.333
>3, 76/78, d1=0.373, d2=0.309 g=3.419
>3, 77/78, d1=0.574, d2=0.423 g=2.488
>3, 78/78, d1=0.558, d2=0.806 g=2.480
>4, 1/78, d1=0.361, d2=0.313 g=2.619
>4, 2/78, d1=0.321, d2=0.327 g=2.777
>4, 3/78, d1=0.332, d2=0.355 g=2.967
>4, 4/78, d1=0.362, d2=0.401 g=2.699
>4, 5/78, d1=0.365, d2=0.508 g=3.143
>4, 6/78, d1=0.515, d2=0.435 g=2.799
>4, 7/78, d1=0.448, d2=0.399 g=2.623
>4, 8/78, d1=0.321, d2=0.271 g=2.990
>4, 9/78, d1=0.399, d2=0.332 g=2.662
>4, 10/78, d1=0.278, 

>6, 45/78, d1=0.275, d2=0.415 g=3.270
>6, 46/78, d1=0.364, d2=0.248 g=3.006
>6, 47/78, d1=0.314, d2=0.366 g=2.986
>6, 48/78, d1=0.333, d2=0.424 g=3.426
>6, 49/78, d1=0.666, d2=0.348 g=2.564
>6, 50/78, d1=0.396, d2=0.462 g=2.993
>6, 51/78, d1=0.450, d2=0.458 g=3.083
>6, 52/78, d1=0.360, d2=0.340 g=3.020
>6, 53/78, d1=0.456, d2=0.315 g=2.935
>6, 54/78, d1=0.402, d2=0.571 g=3.136
>6, 55/78, d1=0.526, d2=0.262 g=2.513
>6, 56/78, d1=0.413, d2=0.704 g=2.636
>6, 57/78, d1=0.356, d2=0.269 g=2.594
>6, 58/78, d1=0.478, d2=0.276 g=2.069
>6, 59/78, d1=0.255, d2=0.706 g=2.571
>6, 60/78, d1=0.325, d2=0.213 g=2.464
>6, 61/78, d1=0.254, d2=0.301 g=2.741
>6, 62/78, d1=0.379, d2=0.354 g=2.570
>6, 63/78, d1=0.362, d2=0.690 g=3.265
>6, 64/78, d1=0.683, d2=0.296 g=2.444
>6, 65/78, d1=0.323, d2=0.488 g=2.708
>6, 66/78, d1=0.407, d2=0.279 g=2.784
>6, 67/78, d1=0.301, d2=0.374 g=2.886
>6, 68/78, d1=0.609, d2=0.462 g=2.743
>6, 69/78, d1=0.300, d2=0.261 g=2.605
>6, 70/78, d1=0.390, d2=0.392 g=2.665
>6, 71/78, d

>9, 28/78, d1=0.305, d2=0.312 g=2.724
>9, 29/78, d1=0.326, d2=0.296 g=2.551
>9, 30/78, d1=0.353, d2=0.424 g=2.694
>9, 31/78, d1=0.362, d2=0.430 g=2.755
>9, 32/78, d1=0.388, d2=0.354 g=2.899
>9, 33/78, d1=0.485, d2=0.463 g=2.907
>9, 34/78, d1=0.332, d2=0.241 g=2.849
>9, 35/78, d1=0.368, d2=0.433 g=2.722
>9, 36/78, d1=0.342, d2=0.298 g=2.706
>9, 37/78, d1=0.267, d2=0.231 g=2.453
>9, 38/78, d1=0.367, d2=0.421 g=2.417
>9, 39/78, d1=0.269, d2=0.447 g=2.595
>9, 40/78, d1=0.453, d2=0.390 g=2.904
>9, 41/78, d1=0.490, d2=0.290 g=2.500
>9, 42/78, d1=0.487, d2=0.659 g=2.351
>9, 43/78, d1=0.407, d2=0.384 g=3.107
>9, 44/78, d1=0.418, d2=0.238 g=2.591
>9, 45/78, d1=0.378, d2=0.313 g=1.865
>9, 46/78, d1=0.238, d2=0.415 g=2.539
>9, 47/78, d1=0.312, d2=0.381 g=2.706
>9, 48/78, d1=0.512, d2=0.406 g=2.465
>9, 49/78, d1=0.443, d2=0.354 g=2.532
>9, 50/78, d1=0.332, d2=0.440 g=2.535
>9, 51/78, d1=0.342, d2=0.332 g=2.916
>9, 52/78, d1=0.459, d2=0.443 g=2.762
>9, 53/78, d1=0.265, d2=0.345 g=2.786
>9, 54/78, d

>12, 6/78, d1=0.362, d2=0.441 g=2.940
>12, 7/78, d1=0.453, d2=0.299 g=2.683
>12, 8/78, d1=0.547, d2=0.459 g=2.489
>12, 9/78, d1=0.257, d2=0.373 g=3.002
>12, 10/78, d1=0.509, d2=0.452 g=2.712
>12, 11/78, d1=0.444, d2=0.380 g=2.647
>12, 12/78, d1=0.289, d2=0.388 g=2.928
>12, 13/78, d1=0.335, d2=0.298 g=3.167
>12, 14/78, d1=0.516, d2=0.289 g=2.722
>12, 15/78, d1=0.264, d2=0.332 g=2.781
>12, 16/78, d1=0.314, d2=0.352 g=3.008
>12, 17/78, d1=0.345, d2=0.422 g=3.092
>12, 18/78, d1=0.340, d2=0.337 g=2.832
>12, 19/78, d1=0.413, d2=0.444 g=3.127
>12, 20/78, d1=0.582, d2=0.371 g=2.719
>12, 21/78, d1=0.352, d2=0.503 g=3.294
>12, 22/78, d1=0.451, d2=0.316 g=2.930
>12, 23/78, d1=0.356, d2=0.327 g=2.668
>12, 24/78, d1=0.353, d2=0.528 g=3.028
>12, 25/78, d1=0.416, d2=0.201 g=2.648
>12, 26/78, d1=0.277, d2=0.390 g=2.557
>12, 27/78, d1=0.402, d2=0.405 g=2.881
>12, 28/78, d1=0.540, d2=0.519 g=3.193
>12, 29/78, d1=0.436, d2=0.274 g=2.947
>12, 30/78, d1=0.487, d2=0.512 g=2.842
>12, 31/78, d1=0.536, d2=0.39

...

>87, 57/78, d1=0.182, d2=0.582 g=3.839
>87, 58/78, d1=0.203, d2=0.312 g=4.515
>87, 59/78, d1=0.400, d2=0.209 g=3.487
>87, 60/78, d1=0.178, d2=0.286 g=3.246
>87, 61/78, d1=0.193, d2=0.229 g=4.063
>87, 62/78, d1=0.258, d2=0.162 g=3.044
>87, 63/78, d1=0.152, d2=0.577 g=4.907
>87, 64/78, d1=0.648, d2=0.179 g=4.100
>87, 65/78, d1=0.211, d2=0.214 g=4.245
>87, 66/78, d1=0.284, d2=0.321 g=4.048
>87, 67/78, d1=0.225, d2=0.201 g=3.689
>87, 68/78, d1=0.208, d2=0.257 g=3.907
>87, 69/78, d1=0.234, d2=0.254 g=4.199
>87, 70/78, d1=0.615, d2=0.385 g=3.543
>87, 71/78, d1=0.159, d2=0.287 g=4.832
>87, 72/78, d1=0.319, d2=0.174 g=3.691
>87, 73/78, d1=0.316, d2=0.413 g=3.868
>87, 74/78, d1=0.152, d2=0.106 g=4.119
>87, 75/78, d1=0.317, d2=0.329 g=3.655
>87, 76/78, d1=0.186, d2=0.260 g=4.018
>87, 77/78, d1=0.332, d2=0.208 g=3.594
>87, 78/78, d1=0.367, d2=0.407 g=3.724
>88, 1/78, d1=0.362, d2=0.139 g=3.337
>88, 2/78, d1=0.153, d2=0.299 g=4.096
>88, 3/78, d1=0.191, d2=0.193 g=4.290
>88, 4/78, d1=0.236, d2=0.13

>90, 34/78, d1=0.387, d2=0.067 g=4.351
>90, 35/78, d1=0.360, d2=0.164 g=2.979
>90, 36/78, d1=0.117, d2=0.376 g=3.676
>90, 37/78, d1=0.216, d2=0.146 g=3.953
>90, 38/78, d1=0.240, d2=0.264 g=4.168
>90, 39/78, d1=0.229, d2=0.269 g=3.956
>90, 40/78, d1=0.376, d2=0.245 g=3.654
>90, 41/78, d1=0.161, d2=0.324 g=3.525
>90, 42/78, d1=0.304, d2=0.327 g=3.838
>90, 43/78, d1=0.228, d2=0.245 g=3.768
>90, 44/78, d1=0.217, d2=0.382 g=4.626
>90, 45/78, d1=0.378, d2=0.243 g=3.930
>90, 46/78, d1=0.324, d2=0.295 g=3.748
>90, 47/78, d1=0.417, d2=0.332 g=3.831
>90, 48/78, d1=0.355, d2=0.328 g=3.535
>90, 49/78, d1=0.142, d2=0.139 g=3.647
>90, 50/78, d1=0.221, d2=0.246 g=3.629
>90, 51/78, d1=0.218, d2=0.265 g=4.408
>90, 52/78, d1=0.222, d2=0.168 g=3.955
>90, 53/78, d1=0.228, d2=0.539 g=5.032
>90, 54/78, d1=0.464, d2=0.160 g=4.053
>90, 55/78, d1=0.230, d2=0.395 g=3.826
>90, 56/78, d1=0.176, d2=0.303 g=4.293
>90, 57/78, d1=0.273, d2=0.338 g=4.397
>90, 58/78, d1=0.321, d2=0.302 g=4.051
>90, 59/78, d1=0.524, d2=

>93, 10/78, d1=0.166, d2=0.365 g=4.535
>93, 11/78, d1=0.388, d2=0.306 g=3.901
>93, 12/78, d1=0.141, d2=0.247 g=4.352
>93, 13/78, d1=0.367, d2=0.178 g=3.402
>93, 14/78, d1=0.275, d2=0.410 g=3.301
>93, 15/78, d1=0.237, d2=0.447 g=4.118
>93, 16/78, d1=0.226, d2=0.128 g=4.059
>93, 17/78, d1=0.252, d2=0.270 g=4.192
>93, 18/78, d1=0.274, d2=0.222 g=3.831
>93, 19/78, d1=0.180, d2=0.162 g=4.133
>93, 20/78, d1=0.230, d2=0.271 g=4.315
>93, 21/78, d1=0.411, d2=0.235 g=4.081
>93, 22/78, d1=0.224, d2=0.229 g=4.309
>93, 23/78, d1=0.188, d2=0.264 g=4.275
>93, 24/78, d1=0.325, d2=0.264 g=3.747
>93, 25/78, d1=0.289, d2=0.323 g=4.025
>93, 26/78, d1=0.292, d2=0.175 g=3.615
>93, 27/78, d1=0.151, d2=0.185 g=3.712
>93, 28/78, d1=0.223, d2=0.498 g=4.723
>93, 29/78, d1=0.412, d2=0.098 g=3.221
>93, 30/78, d1=0.170, d2=0.259 g=3.562
>93, 31/78, d1=0.149, d2=0.318 g=4.428
>93, 32/78, d1=0.383, d2=0.353 g=4.422
>93, 33/78, d1=0.316, d2=0.214 g=4.203
>93, 34/78, d1=0.354, d2=0.413 g=4.397
>93, 35/78, d1=0.287, d2=

>95, 65/78, d1=0.179, d2=0.223 g=4.324
>95, 66/78, d1=0.208, d2=0.169 g=4.123
>95, 67/78, d1=0.205, d2=0.405 g=4.097
>95, 68/78, d1=0.218, d2=0.218 g=3.934
>95, 69/78, d1=0.246, d2=0.205 g=4.108
>95, 70/78, d1=0.328, d2=0.555 g=4.914
>95, 71/78, d1=0.523, d2=0.274 g=4.196
>95, 72/78, d1=0.225, d2=0.245 g=4.341
>95, 73/78, d1=0.315, d2=0.243 g=3.393
>95, 74/78, d1=0.118, d2=0.307 g=3.924
>95, 75/78, d1=0.253, d2=0.367 g=4.096
>95, 76/78, d1=0.204, d2=0.100 g=3.861
>95, 77/78, d1=0.158, d2=0.231 g=3.407
>95, 78/78, d1=0.170, d2=0.314 g=4.167
>96, 1/78, d1=0.299, d2=0.124 g=3.598
>96, 2/78, d1=0.275, d2=0.405 g=4.576
>96, 3/78, d1=0.334, d2=0.188 g=4.541
>96, 4/78, d1=0.239, d2=0.301 g=4.128
>96, 5/78, d1=0.220, d2=0.371 g=4.622
>96, 6/78, d1=0.286, d2=0.110 g=4.150
>96, 7/78, d1=0.294, d2=0.296 g=3.485
>96, 8/78, d1=0.228, d2=0.311 g=4.222
>96, 9/78, d1=0.374, d2=0.173 g=3.351
>96, 10/78, d1=0.204, d2=0.218 g=3.230
>96, 11/78, d1=0.083, d2=0.417 g=4.761
>96, 12/78, d1=0.533, d2=0.180 g=3

>98, 42/78, d1=0.319, d2=0.059 g=3.539
>98, 43/78, d1=0.176, d2=0.424 g=3.569
>98, 44/78, d1=0.234, d2=0.166 g=3.747
>98, 45/78, d1=0.374, d2=0.262 g=3.242
>98, 46/78, d1=0.191, d2=0.245 g=3.718
>98, 47/78, d1=0.226, d2=0.425 g=4.250
>98, 48/78, d1=0.207, d2=0.150 g=4.004
>98, 49/78, d1=0.235, d2=0.416 g=4.744
>98, 50/78, d1=0.611, d2=0.351 g=3.837
>98, 51/78, d1=0.117, d2=0.302 g=4.525
>98, 52/78, d1=0.234, d2=0.183 g=4.111
>98, 53/78, d1=0.140, d2=0.247 g=3.779
>98, 54/78, d1=0.313, d2=0.435 g=3.831
>98, 55/78, d1=0.174, d2=0.174 g=4.539
>98, 56/78, d1=0.286, d2=0.255 g=3.908
>98, 57/78, d1=0.270, d2=0.165 g=3.930
>98, 58/78, d1=0.245, d2=0.497 g=4.759
>98, 59/78, d1=0.290, d2=0.150 g=4.211
>98, 60/78, d1=0.315, d2=0.341 g=3.877
>98, 61/78, d1=0.105, d2=0.247 g=4.706
>98, 62/78, d1=0.359, d2=0.304 g=4.098
>98, 63/78, d1=0.165, d2=0.153 g=4.101
>98, 64/78, d1=0.290, d2=0.222 g=4.195
>98, 65/78, d1=0.247, d2=0.159 g=3.710
>98, 66/78, d1=0.202, d2=0.295 g=4.615
>98, 67/78, d1=0.356, d2=

In [10]:
d_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 80, 80, 128)       9728      
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 80, 80, 128)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 40, 40, 128)       409728    
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 40, 40, 128)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 20, 20, 128)       409728    
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 20, 20, 128)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 10, 10, 128)       4

In [9]:
g_model.layers[0]

<module 'matplotlib.pyplot' from 'C:\\ProgramData\\Anaconda3\\lib\\site-packages\\matplotlib\\pyplot.py'>